# F.E.R.B – Fine-tuned Encoder for Response and Behavior

## Step 1: Installing and importing the libraries

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 121.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.

In [ ]:
!pip install huggingface_hub

In [ ]:
!pip install --upgrade bitsandbytes
!pip install --upgrade transformers accelerate
!pip install --upgrade trl peft datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.0 MB/s eta 0:00:00
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.40.2
    Uninstalling bitsandbytes-0.40.2:
      Successfully uninstalled bitsandbytes-0.40.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 40.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0
  Attempting uninstall: accelerate
    Found existing installation: accelerat

In [ ]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline)

## Step 2: Loading the model

In [ ]:
llama_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v2",
                                                   quantization_config = BitsAndBytesConfig(load_in_4bit = True,
                                                                                            bnb_4bit_compute_dtype = getattr(torch, "float16"),
                                                                                            bnb_4bit_quant_type = "nf4"))
llama_model.config.use_cache = False
llama_model.config.pretraining_tp = 1

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

## Step 3: Loading the tokenizer

In [ ]:
llama_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v2", trust_remote_code = True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

## Step 4: Setting the training arguments

In [ ]:
training_arguments = TrainingArguments(output_dir = "./results", per_device_train_batch_size = 4, max_steps = 100)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi


Mon Feb 10 12:11:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P0             29W /   70W |    4314MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!kill -9 $(nvidia-smi | awk '$2=="Processes:" {f=1; next} f {print $5}' | xargs)


/bin/bash: line 1: kill: PID: arguments must be process or job IDs
/bin/bash: line 1: kill: |: arguments must be process or job IDs


In [ ]:
!nvidia-smi


Mon Feb 10 12:11:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P0             29W /   70W |    4314MiB /  15360MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:

torch.cuda.empty_cache()


## Step 5: Creating the Supervised Fine-Tuning trainer

In [ ]:
# ✅ Define Training Arguments with Fewer Steps
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,  # ✅ Reduce epochs further if needed
    max_steps=30,  # ✅ Stop training after 500 steps (Adjust as needed)
    per_device_train_batch_size=1,  # ✅ Lower batch size to avoid OOM
    gradient_accumulation_steps=8,  # ✅ Simulates larger batch
    fp16=True,  # ✅ Mixed precision training
    save_steps=100,  # ✅ Save model every 100 steps
    save_total_limit=2,
    logging_steps=10,
    report_to="wandb"  # ✅ Logs to Weights & Biases
)

# ✅ Initialize Trainer
llama_sft_trainer = SFTTrainer(
    model=llama_model,
    args=training_arguments,
    train_dataset=train_dataset,
    peft_config=peft_config
)


/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(


## Step 6: Training the model

fdd17058c6cc236455c7169f3ebaebfc2628d1aa

In [ ]:
llama_sft_trainer.train()

Step,Training Loss
10,5.429400
20,3.872200
30,2.821000


TrainOutput(global_step=30, training_loss=4.040852355957031, metrics={'train_runtime': 418.583, 'train_samples_per_second': 0.573, 'train_steps_per_second': 0.072, 'total_flos': 4896201312829440.0, 'train_loss': 4.040852355957031, 'epoch': 0.034980323567993})

## Step 7: Chatting with the model

In [ ]:
State = True
user_prompt = input()
while State == True:
  #user_prompt = input()
  if user_prompt.lower()== 'hey ferb':
    query = input()
    if query.lower() == "bye ferb":
      print("F.E.R.B: Bye!")
      State = False
      break
    else:
      text_generation_pipeline = pipeline(task = "text-generation", model = llama_model, tokenizer = llama_tokenizer, max_length = 300)
      model_answer = text_generation_pipeline(f"<s>[INST] {query} [/INST]")
      print('F.E.R.B: ',model_answer[0]['generated_text'])
  else:
    print("Type Hey Ferb")



hey ferb
fever


Device set to use cuda:0


F.E.R.B:  <s>[INST] fever [/INST]  Fever is a common medical condition characterized by an elevated body temperature, typically above 100. nobody knows for sure why fever occurs, but it is believed to be a response of the body's immune system to an infection or inflammation.

The body's normal temperature range is between 97.7°F (36.5°C) and 99.5°F (37.5°C). When the body detects the presence of an infection or inflammation, it releases chemicals called cytokines, which cause the body to produce heat. This heat is produced by the muscles, liver, and other organs, and it is released into the bloodstream as heat.

There are several ways to measure a fever, including:

1. Oral thermometer: This is the most common method of measuring a fever. It involves placing a thermometer under the tongue and waiting for the temperature to stabilize.
2. Rectal thermometer: This method is more accurate than oral thermometry, as the temperature is taken directly from the rectum.
3. Axillary thermometer: 

Device set to use cuda:0


F.E.R.B:  <s>[INST] I have panic attacks. What are the causes? [/INST]  Panic attacks are a common symptom of panic disorder, a mental health condition that can cause feelings of intense fear or anxiety. The causes of panic attacks can vary, but some common triggers include:

1. Genetics: Panic disorder can run in families, suggesting that there may be a genetic component to the condition.
2. Brain chemistry: Imbalances in neurotransmitters such as serotonin and gamma-aminobutyric acid (GABA) can contribute to panic attacks.
3. Stress: Traumatic events, such as the loss of a loved one or a job, can trigger panic attacks in people who are prone to the condition.
4. Anxiety: People with anxiety disorders, such as generalized anxiety disorder or social anxiety disorder, are more likely to experience panic attacks.
5. Medical conditions: Certain medical conditions, such as thyroid disorders or heart conditions, can cause panic attacks.
6. Substance use: Withdrawal from certain drugs, such 